In [1]:
import pandas as pd

In [ ]:
import json
import csv
from datetime import datetime

# Load JSON file
with open('../data/foodkeeper.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Extract Categories (ID → Name mapping)
categories = {}
for sheet in data['sheets']:
    if sheet['name'] == 'Category':
        for row in sheet['data']:
            category_id = row[0]['ID']
            category_name = row[1]['Category_Name']
            categories[category_id] = category_name
        break

# Extract Product data
products = []
for sheet in data['sheets']:
    if sheet['name'] == 'Product':
        for row in sheet['data']:
            product = {}
            for field in row:
                key = list(field.keys())[0]
                product[key] = field[key]
            
            # Skip if no name
            if 'Name' not in product:
                continue
            
            # Get category name
            category_id = product.get('Category_ID')
            category = categories.get(category_id, "Unknown")
            
            # Determine shelf life (prioritize Pantry → Refrigerate → Freeze)
            shelf_life = None
            metric = None
            
            # Check Pantry first
            if product.get('Pantry_Min') is not None:
                shelf_life = f"{product['Pantry_Min']}-{product['Pantry_Max']} {product['Pantry_Metric']} (Pantry)"
            # Else check Refrigerator
            elif product.get('Refrigerate_Min') is not None:
                shelf_life = f"{product['Refrigerate_Min']}-{product['Refrigerate_Max']} {product['Refrigerate_Metric']} (Refrigerator)"
            # Else check Freezer
            elif product.get('Freeze_Min') is not None:
                shelf_life = f"{product['Freeze_Min']}-{product['Freeze_Max']} {product['Freeze_Metric']} (Freezer)"
            
            # Skip if no shelf life data
            if not shelf_life:
                continue
            
            # Add to products list
            products.append({
                'Name': product['Name'],
                'Category': category,
                'Shelf_Life': shelf_life,
            })
        break

output_filename = f"../data_processed/foodkeeper_shelf_life_processed.csv"

# Write to CSV
with open(output_filename, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=['Name', 'Category', 'Shelf_Life'])
    writer.writeheader()
    writer.writerows(products)

print(f"CSV generated: {output_filename}")
print(f"Total products processed: {len(products)}")

CSV generated: foodkeeper_shelf_life_processed.csv
Total products processed: 249
